In [1]:
import numpy as np
import pandas as pd
import pathlib
from tqdm.auto import tqdm

import hydra
from omegaconf import DictConfig, OmegaConf

import torch
#from torch_geometric import seed_everything

import ray

In [2]:
node = !hostname
if "sc" in node[0]:
    base_path = "/sc-projects/sc-proj-ukb-cvd"
else: 
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"
print(base_path)

project_label = "22_retina_phewas"
project_path = f"{base_path}/results/projects/{project_label}"
figure_path = f"{project_path}/figures"
output_path = f"{project_path}/data"

pathlib.Path(figure_path).mkdir(parents=True, exist_ok=True)
pathlib.Path(output_path).mkdir(parents=True, exist_ok=True)

##### BEGIN ADAPT #####
# second best model
# wandb_name = 'aug++_convnext_s_mlp'
# wandb_id = '8ngm6apd'
# best model
wandb_name = 'aug++_convnext_s_mlp+'
wandb_id = '3p3smraz'
##### END   ADAPT #####

feather_path = f'/sc-projects/sc-proj-ukb-cvd/results/models/retina/{wandb_id}/checkpoints/predictions/predictions.feather'
experiment = wandb_id
experiment_path = f"{output_path}/{experiment}"
pathlib.Path(experiment_path).mkdir(parents=True, exist_ok=True)

/sc-projects/sc-proj-ukb-cvd


In [3]:
endpoint_columns = sorted([l.replace('_prevalent', '') for l in list(pd.read_csv('/sc-projects/sc-proj-ukb-cvd/results/projects/22_medical_records/data/220413/frequent_endpoints.csv').endpoint.values)])

In [4]:
ray.shutdown()
ray.init(num_cpus=24)

{'node_ip_address': '10.32.105.6',
 'raylet_ip_address': '10.32.105.6',
 'redis_address': '10.32.105.6:6379',
 'object_store_address': '/tmp/ray/session_2022-05-25_11-16-49_389648_3762942/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2022-05-25_11-16-49_389648_3762942/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2022-05-25_11-16-49_389648_3762942',
 'metrics_export_port': 59233,
 'node_id': 'c64a891f4c1c291fef8af6c916df728933d2af28848d42389793f580'}

In [5]:
import wandb
api = wandb.Api()
entity, project = "cardiors", "retina"  # 'll-cha'
runs = api.runs(entity + "/" + project) 

In [6]:
run_list = []
for run in tqdm(runs): 
    run_list.append(
        {
            "id": run.id, #path[-1]
            "name": run.name,
            "tags": run.tags,
            "config": {k: v for k,v in run.config.items() if not k.startswith('_')},
            "summary": run.summary._json_dict,
            #"path": None if "predictions_path" not in run.config.keys() else str(pathlib.Path(run.config["predictions_path"]))
            'path': f'/sc-projects/sc-proj-ukb-cvd/results/models/retina/{run.id}/checkpoints/predictions/predictions.feather'
        }
    )

  0%|          | 0/114 [00:00<?, ?it/s]

In [7]:
runs_df = pd.DataFrame(run_list)
print(runs_df['path'].iloc[0])

/sc-projects/sc-proj-ukb-cvd/results/models/retina/3h7yt93y/checkpoints/predictions/predictions.feather


In [8]:
tag = experiment # or 'your_experiment_tag'
# runs_df = runs_df[runs_df.tags.astype(str).str.contains(tag)].query("path==path")
runs_df = runs_df[runs_df.id.astype(str).str.contains(tag)].query("path==path")
runs_df

,id,name,tags,config,summary,path
22,3p3smraz,aug++_convnext_s_mlp+,"[baseline_data, image]",{'losses': ['<retinalrisk.models.loss_wrapper....,{'valid/phecode_144-2 - Benign neoplasms of th...,/sc-projects/sc-proj-ukb-cvd/results/models/re...


## Process Predictions

In [9]:
name_dict = {
    "CovariatesOnlyTraining_['age_at_recruitment_f21022_0_0', 'sex_f31_0_0']_None_MLPHead": "Identity(AgeSex)+MLP",
    "RetinaIdentityTraining_[]_None_MLPHead": "Identity(Retina)+MLP",
    #"RecordsGraphTraining_[]_HeteroGNN_MLPHead": "GNN(Records)+MLP",
    #"RecordsIdentityTraining_['age_at_recruitment_f21022_0_0', 'sex_f31_0_0']_None_MLPHead": "Identity(AgeSex+Records)+MLP",
    #"RecordsGraphTraining_['age_at_recruitment_f21022_0_0', 'sex_f31_0_0']_HeteroGNN_MLPHead": "GNN(AgeSex+Records)+MLP"
}

In [10]:
id_vars = ["eid", "model", "partition", "split"]

In [11]:
out_path = f"{experiment_path}/loghs"
pathlib.Path(out_path).mkdir(parents=True, exist_ok=True)

In [12]:
@ray.remote
def prepare_predictions(in_path, out_path):
    temp = pd.read_feather(in_path).rename(columns={"index": "eid"}).set_index('eid')
    # mean duplicated indices aka left and right eye value per eid
    temp_with_meaned_test_preds = temp.groupby(level=0).mean()
    for col in temp.columns.values:
        if col not in endpoint_columns:
            temp_with_meaned_test_preds[col] = temp[col].iloc[~temp.index.duplicated()].values
    temp = temp_with_meaned_test_preds.reset_index(drop=False)
    
    # rename 10_1_Ft__ and dropping ft and St cols if present
    cols_to_drop = []
    cols_to_rename = {}
    for col in temp.columns.values:
        if 'ft' in col:
            cols_to_drop.append(col)
        elif 'St' in col:
            cols_to_drop.append(col)
        elif 'Ft' in col:
            cols_to_rename[col] = col.replace('1_10_Ft__', '')
            
    temp['record_cols'] = None

    temp["model"] = (temp.module.astype(str) + "_" + temp.covariate_cols.astype(str) + "_" + temp.encoder.astype(str) + "_" + temp["head"].astype(str)).astype("category")
    temp = temp.replace({"model":name_dict}).drop(columns=["module", "encoder", "head", "covariate_cols", "record_cols"]).drop(columns=cols_to_drop).rename(columns=cols_to_rename)
    for c in id_vars: temp[c] = temp[c].astype("category")
    
    model = temp.model.unique()[0]
    partition = temp.partition.unique()[0]
    for split in ["train", "valid", "test"]: #"test_left", 'test_right'
        fp_out = f"{out_path}/{model}/{partition}"
        pathlib.Path(fp_out).mkdir(parents=True, exist_ok=True)
        t = temp.query("split==@split")
        t.reset_index(drop=True).to_feather(f"{fp_out}/{split}.feather")

In [13]:
for row_idx in tqdm(range(len(runs_df))): 
    row = runs_df.iloc[row_idx]
    p = row['path']
    id = row['id']
    prepare_predictions.remote(p, out_path)

  0%|          | 0/1 [00:00<?, ?it/s]

In [14]:
out_path

'/sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas/data/3p3smraz/loghs'

In [15]:
!ls /sc-projects/sc-proj-ukb-cvd/results/projects/22_retina_phewas/data/test_experiment/loghs/

MultiTaskSurvivalTraining_None_InceptionResnetV2+MLP_Identity
